In [1]:
%pwd

'f:\\Git Projects\\Venkey-Chicken-Disease-Classification\\research'

In [2]:
import os
os.chdir("../")
%pwd

'f:\\Git Projects\\Venkey-Chicken-Disease-Classification'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [4]:
from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])
        return PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath) # .keras
        )

In [6]:
import tensorflow as tf
import time

In [7]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True,
            monitor="val_loss",
            mode="min",
            verbose=1
        )

    @property
    def _create_early_stopping(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=5,
            restore_best_weights=True,
            verbose=1
        )

    @property
    def _create_reduce_lr(self):
        return tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=2,
            min_lr=1e-6,
            verbose=1
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
            self._create_early_stopping,
            self._create_reduce_lr
        ]


In [8]:
try:
    cfg = ConfigurationManager()
    cb_cfg = cfg.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=cb_cfg)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    print("Prepared callbacks:", [type(cb).__name__ for cb in callback_list])

except Exception as e:
    raise e

[2025-08-16 10:38:23,538: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-16 10:38:23,541: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-16 10:38:23,543: INFO: common: created directory at: artifacts]
[2025-08-16 10:38:23,545: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-08-16 10:38:23,547: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
Prepared callbacks: ['TensorBoard', 'ModelCheckpoint', 'EarlyStopping', 'ReduceLROnPlateau']


In [9]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2025-08-16-10-38-34'